In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd /gdrive/My\ Drive/nn 

/gdrive/My Drive/nn


In [0]:
%tensorflow_version 1.x
import argparse
import os
import pickle
import sys

import numpy as np

np.random.seed(7)  # for reproducibility

import tensorflow as tf
tf.random.set_random_seed(5005)

from sklearn.model_selection import train_test_split, KFold


from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.python.keras.backend as K
from tensorflow.keras.utils import to_categorical
from keras import regularizers

import matplotlib as mpl

mpl.use('Agg')
import utils
sys.path.append(".")

import importlib
importlib.reload(utils)


from utils import precision, recall_TP, load_data_merged, recall_TN



l2_lam = 5e-07 
l1_lam = 1e-08 


In [0]:
def train_diff_model(data_path, res_path, model_name, input_len,
                     num_epoch, batchsize, func,model_path="./weights.hdf5", 
                     number_of_folds=1, save=True):
    """
    Training the model
    :param data_path: path to file (consist of train, valid and test data)
    :param res_path:
    :param model_name:
    :param input_len:
    :param num_epoch:
    :param batchsize:
    :param model_path:
    :return:
    """
    print('creating model')
    model = func(input_len)
    print('compiling model')
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', recall_TN, recall_TP])
    checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    print('loading data')
    x_train_list, y_train_list, x_valid_list, y_valid_list, x_test_seq, y_test = load_data_merged(data_path, input_len, kfold=number_of_folds)

    print('fitting the model')
    for i in range(len(x_train_list)):
      print("Using fold %s/%s" %(i+1, number_of_folds))
      x_train_seq = x_train_list[i]
      y_train = y_train_list[i]
      x_valid_seq = x_valid_list[i]
      y_valid = y_valid_list[i]

      history = model.fit(x_train_seq, y_train, epochs=num_epoch, batch_size=batchsize,
                          validation_data=(x_valid_seq, y_valid), verbose=1,
                          callbacks=[checkpointer, earlystopper, ])  # tb])
    
    print("Finish training")
    if save:
      print('saving the model')
      model.save(os.path.join(res_path, model_name + ".h5"))

    print('testing the model')
    score = model.evaluate(x_test_seq, y_test)

    for i in range(len(model.metrics_names)):
        print(str(model.metrics_names[i]) + ": " + str(score[i]))

In [0]:
train_diff_model(data_path=r"dataset/solo_test_with_crc10_var_0.05.pkl", 
                 res_path="./models", model_name="150cpg", model_path="./models/temp/2",
                 input_len=150, num_epoch=20, batchsize=128, number_of_folds=2,save=False, func = model26_avg_pool)

**Models**

In [0]:
def model_baseline_with_drop(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv1 = Conv1D(filters=90, kernel_size=7, padding='valid', activation="relu", name="conv1")(input_node)
    pool1 = MaxPooling1D(pool_size=4, strides=2, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2 = MaxPooling1D(pool_size=10, strides=5)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1")(flat)
    drop3 = Dropout(0.25)(hidden1)
    output = Dense(1, activation='sigmoid', name="output")(drop3)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [0]:
def model_baseline_with_smaller_conv(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1")(input_node)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2 = MaxPooling1D(pool_size=10, strides=5)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1")(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [0]:
def model_baseline_with_conv_1(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0")(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1")(conv0)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2 = MaxPooling1D(pool_size=10, strides=5)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1")(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [0]:
def model_baseline_after_30_with_3conv(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0")(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1")(conv0)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2 = MaxPooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)

    conv3 = Conv1D(filters=100, kernel_size=7, padding='valid', activation="relu", name="conv3")(drop2)
    pool3 = MaxPooling1D(pool_size=4, strides=2)(conv3)
    drop3 = Dropout(0.25)(pool3)
    flat = Flatten()(drop3)

    hidden1 = Dense(500, activation='relu', name="hidden1")(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [0]:
def model_baseline_with_conv_1_and_smaller_decrease_second_pool(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0")(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1")(conv0)
    pool1 = MaxPooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2 = MaxPooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1")(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [0]:
from tensorflow.python.keras.layers.pooling import AveragePooling1D

def model26_avg_pool(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv0 = Conv1D(filters=90, kernel_size=1, padding='valid', activation="relu", name="conv0",kernel_regularizer=regularizers.l2(l2_lam))(input_node)
    conv1 = Conv1D(filters=90, kernel_size=3, padding='valid', activation="relu", name="conv1",kernel_regularizer=regularizers.l2(l2_lam))(conv0)
    pool1 = AveragePooling1D(pool_size=2, strides=1, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2",kernel_regularizer=regularizers.l2(l2_lam))(drop1)
    pool2 = AveragePooling1D(pool_size=2, strides=1)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1",kernel_regularizer=regularizers.l1(l1_lam))(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

In [0]:
def deepripe_model(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv1 = Conv1D(filters=90, kernel_size=7, padding='valid', activation="relu", name="conv1")(input_node)
    pool1 = MaxPooling1D(pool_size=4, strides=2, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)

    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2  = MaxPooling1D(pool_size=10, strides=5)(conv2)
    drop2  = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(250, activation='relu', name="hidden1")(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model



In [0]:
def model_baseline(input_len):
    K.clear_session()
    # tf.random.set_random_seed(5005)

    input_node = Input(shape=(input_len, 4), name="input")
    conv1 = Conv1D(filters=90, kernel_size=7, padding='valid', activation="relu", name="conv1")(input_node)
    pool1 = MaxPooling1D(pool_size=4, strides=2, name="pool1")(conv1)
    drop1 = Dropout(0.25, name="drop1")(pool1)
  
    conv2 = Conv1D(filters=100, kernel_size=5, padding='valid', activation="relu", name="conv2")(drop1)
    pool2 = MaxPooling1D(pool_size=10, strides=5)(conv2)
    drop2 = Dropout(0.25)(pool2)
    flat = Flatten()(drop2)

    hidden1 = Dense(500, activation='relu', name="hidden1")(flat)
    output = Dense(1, activation='sigmoid', name="output")(hidden1)
    model = Model(inputs=[input_node], outputs=output)
    print(model.summary())
    return model

Play
